## Prerequisites

1. Complete a Standard agent setup.
2. Ensure that you have the role **Storage Blob Data Contributor** on your project's storage account.
3. Ensure that you have the role **Azure AI Developer** on your project.


## Setup

In [ ]:
# Install dependencies

%pip install azure-identity
%pip install azure-ai-projects
%pip install python-dotenv

In [ ]:
# Load environment

from dotenv import load_dotenv

load_dotenv(override=True)

In [ ]:
# Create the project client

import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
project_client = AIProjectClient.from_connection_string(
    credential=credential, conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

## Setup the Agent

For this, we're pre-uploading the file to so that the project is aware of the file. After the file has been uploaded, we create a new vector store that includes that project file.

### Create the Vector Store

In [ ]:
# Upload files and add them to Vector Store

from azure.ai.projects.models import FilePurpose

# Upload...
file = project_client.agents.upload_file_and_poll(
    file_path='./assets/MSFT_cloud_architecture_contoso.pdf',
    purpose=FilePurpose.AGENTS
)
print(f"Uploaded file, file ID: {file.id}")

# Create a vector store with the files you uploaded
vector_store = project_client.agents.create_vector_store_and_poll(
    file_ids=[file.id],
    name="agent-vectorstore"
)
print(f"Created vector store, vector store ID: {vector_store.id}")

### Create the Agent to use the agent-level Vector Store

In [ ]:
# Create an agent and enable file search

from azure.ai.projects.models import FileSearchTool

file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

agent = project_client.agents.create_agent(
    model="gpt-4o-mini",
    name="agent-test",
    instructions="You are a helpful agent",
    tools=file_search_tool.definitions,
    tool_resources=file_search_tool.resources
)

print(f"Created agent, agent ID: {agent.id}")

## Run it!

### Create a thread that only uses the agent-level vector store

In [ ]:
# Create a thread - ask a question over an existing vector store
# Agent has access because the tool was added at the agent, and
# it included a reference to the vector store that was created
# for the pre-uploaded files

from azure.ai.projects.models import MessageTextContent

thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {thread.id}")

# Add a user message to the thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="What compute resources are being considered by Contoso?"
)

# Run the agent and wait for a response
run = project_client.agents.create_and_process_run(
    thread_id=thread.id,
    agent_id=agent.id
)
print(f"Completed run, run ID: {run.id}")

In [ ]:
# Print run results
messages = project_client.agents.list_messages(thread_id=thread.id)
for m in reversed(messages.data):
    last_message_content = m.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"\n{m.role.upper()}: {last_message_content.text.value}")

print("\nCitations:")
for c in messages.file_citation_annotations:
    print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")

### Create a thread that uses both the agent-level vector store ***and*** a thread-level vector store (from attachments)

In [ ]:
# Create a thread - ask a question including a new file
# Agent has access to the vector store because of the original initialization
# of the agent. Another vector store is created for the attachment knowledge
# only.

from azure.ai.projects.models import MessageAttachment

attachments_thread = project_client.agents.create_thread()
print(f"Created thread, thread ID: {attachments_thread.id}")

# Upload a new file as a message attachment
message_file = project_client.agents.upload_file_and_poll(file_path="./assets/contoso-checklist.md", purpose=FilePurpose.AGENTS)
print(f"Uploaded file, file ID: {message_file.id}")

# Create a message with the file search attachment
# Notice that vector store is created temporarily when using attachments with a default expiration policy of seven days.
attachment = MessageAttachment(file_id=message_file.id, tools=FileSearchTool().definitions)
attachments_message = project_client.agents.create_message(
    thread_id=attachments_thread.id,
    role="user",
    content="What compute resources are targeted?",
    attachments=[attachment]
)
print(f"Created message, message ID: {message.id}")

# Run the thread
attachments_run = project_client.agents.create_and_process_run(
    thread_id=attachments_thread.id,
    agent_id=agent.id
)
print(f"Completed run, run ID: {attachments_run.id}")

# Create another message that requires knowledge from original document
another_attachments_message = project_client.agents.create_message(
    thread_id=attachments_thread.id,
    role="user",
    content="Where is Contoso headquartered?"
)
print(f"Created message, message ID: {another_attachments_message.id}")

# Run the thread
another_attachments_run = project_client.agents.create_and_process_run(
    thread_id=attachments_thread.id,
    agent_id=agent.id
)
print(f"Completed run, run ID: {another_attachments_run.id}")

In [ ]:
# Print results

attachments_messages = project_client.agents.list_messages(thread_id=attachments_thread.id)
for m in reversed(attachments_messages.data):
    last_message_content = m.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"\n{m.role.upper()}: {last_message_content.text.value}")

print("\nCitations:")
for c in attachments_messages.file_citation_annotations:
    print(f"- Citation snippet: '{c.text}' from file ID: {c.file_citation['file_id']}")

## Cleanup Agent, Files, and Vector Store

In [ ]:
# Explicit cleanup
project_client.agents.delete_vector_store(vector_store_id=vector_store.id)
project_client.agents.delete_file(file_id=file.id)
project_client.agents.delete_file(file_id=message_file.id)
project_client.agents.delete_thread(thread_id=thread.id)
project_client.agents.delete_thread(thread_id=attachments_thread.id)
project_client.agents.delete_agent(agent_id=agent.id)

# Clean up EVERYTHING...

# # Cleanup any vector stores
# vector_stores = project_client.agents.list_vector_stores()
# for vs in vector_stores.data:
#     print(f"Deleting vector store: {vs.id}")
#     project_client.agents.delete_vector_store(vector_store_id=vs.id)
# # Cleanup any files
# files = project_client.agents.list_files(purpose=FilePurpose.AGENTS)
# for f in files.data:
#     print(f"Deleting file: {f.id}")
#     project_client.agents.delete_file(file_id=f.id)
# # Cleanup any agents
# agents = project_client.agents.list_agents()
# for a in agents.data:
#     print(f"Deleting agent: {a.id}")
#     project_client.agents.delete_agent(agent_id=a.id)